<a href="https://colab.research.google.com/github/BrahiamAcosta/Competencia-Kaggle-Modelos1/blob/main/04%20-%20modelo%20con%20preprocesado%20categorico%20imputado%20con%20TargetEncoding%20y%20Catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Carga de datos

In [ ]:
import pandas as pd
import os

os.environ["KAGGLE_CONFIG_DIR"] = "."


!kaggle competitions download -c udea-ai4eng-20242
!unzip udea-ai4eng-20242.zip
!rm udea-ai4eng-20242.zip

 84% 17.0M/20.1M [00:00<00:00, 38.6MB/s]
100% 20.1M/20.1M [00:00<00:00, 39.7MB/s]
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


##Limpieza preliminar de los datos

In [ ]:
!pip install category_encoders
!pip install catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [ ]:
#Cleaning dataset function
def clean_dataset(df):
  matricula = {
    'No pagó matrícula': 0,
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7
  }
  estratos_vivienda = {
      'Estrato 1': 1,
      'Estrato 2': 2,
      'Estrato 3': 3,
      'Estrato 4': 4,
      'Estrato 5': 5,
      'Estrato 6': 6,
      'Sin Estrato': 0
  }
  educacion_padres = {
      'Ninguno': 1,
      'Primaria incompleta': 2,
      'Primaria completa': 3,
      'Secundaria (Bachillerato) incompleta': 4,
      'Secundaria (Bachillerato) completa': 5,
      'Técnica o tecnológica incompleta': 6,
      'Técnica o tecnológica completa': 7,
      'Educación profesional incompleta': 8,
      'Educación profesional completa': 9,
      'Postgrado': 10,
      'No sabe': 0,
      'No Aplica': -1,
  }
  internet = {
      'Si': 1,
      'No': 0
  }
  pago_matricula = {
      'Si': 1,
      'No': 0
  }
  trabaja ={
      '0':0,
      'Menos de 10 horas':1,
      'Entre 11 y 20 horas':2,
      'Entre 21 y 30 horas':3,
      'Más de 30 horas':4
  }
  if 'RENDIMIENTO_GLOBAL' in df.columns:
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map({"bajo":1, "medio-bajo":2, "medio-alto":3, "alto":4})
  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(matricula)
  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(-1)
  df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(estratos_vivienda)
  df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna(-1)
  df.FAMI_EDUCACIONMADRE = df.FAMI_EDUCACIONMADRE.map(educacion_padres)
  df.FAMI_EDUCACIONPADRE = df.FAMI_EDUCACIONPADRE.map(educacion_padres)
  df.FAMI_EDUCACIONMADRE = df.FAMI_EDUCACIONMADRE.fillna(0)
  df.FAMI_EDUCACIONPADRE = df.FAMI_EDUCACIONPADRE.fillna(0)
  df.FAMI_TIENEINTERNET = df.FAMI_TIENEINTERNET.map(internet)
  df.FAMI_TIENEINTERNET = df.FAMI_TIENEINTERNET.fillna(-1)
  df.ESTU_PAGOMATRICULAPROPIO = df.ESTU_PAGOMATRICULAPROPIO.map(pago_matricula)
  df.ESTU_PAGOMATRICULAPROPIO = df.ESTU_PAGOMATRICULAPROPIO.fillna(-1)
  df.ESTU_HORASSEMANATRABAJA = df.ESTU_HORASSEMANATRABAJA.map(trabaja)
  df.ESTU_HORASSEMANATRABAJA = df.ESTU_HORASSEMANATRABAJA.fillna(-1)

  return df

In [ ]:
dtr = pd.read_csv("train.csv")
del(dtr['ID'])
dtr.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [ ]:
dtr = clean_dataset(dtr)
dtr.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,20212,ENFERMERIA,BOGOTÁ,6.0,1.0,3.0,1.0,6.0,10.0,0.0,3
1,20212,DERECHO,ATLANTICO,4.0,0.0,3.0,0.0,7.0,6.0,0.0,1
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,4.0,4.0,3.0,1.0,5.0,5.0,0.0,1
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,5.0,0.0,4.0,1.0,0.0,5.0,0.0,4
4,20212,PSICOLOGIA,ANTIOQUIA,4.0,3.0,3.0,1.0,3.0,3.0,0.0,2


In [ ]:
#Encoder
from category_encoders import MEstimateEncoder as ME
cols_to_encode = ['PERIODO',	'ESTU_PRGM_ACADEMICO'	,'ESTU_PRGM_DEPARTAMENTO']
encoder = ME(m=40)
encoder.fit(dtr[cols_to_encode].astype(str), dtr['RENDIMIENTO_GLOBAL'])

def transform_data(df,transformer=encoder):
  df_encoded = encoder.transform(df[cols_to_encode].astype(str))
  df_final = pd.concat([df_encoded,df.drop(cols_to_encode, axis=1)],axis=1)
  return df_final

In [ ]:
dtr = transform_data(dtr)
dtr.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,2.394607,2.314279,2.557373,6.0,1.0,3.0,1.0,6.0,10.0,0.0,3
1,2.394607,2.629367,2.454827,4.0,0.0,3.0,0.0,7.0,6.0,0.0,1
2,2.584049,2.480372,2.557373,4.0,4.0,3.0,1.0,5.0,5.0,0.0,1
3,2.458686,2.298972,2.733969,5.0,0.0,4.0,1.0,0.0,5.0,0.0,4
4,2.394607,2.466352,2.667522,4.0,3.0,3.0,1.0,3.0,3.0,0.0,2


In [ ]:
X = dtr.drop("RENDIMIENTO_GLOBAL", axis=1)
y = dtr.RENDIMIENTO_GLOBAL

In [ ]:
#Modelo RandomForest
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split, RandomizedSearchCV

z=CatBoostClassifier(
    iterations=500
)

z.fit(X,y)

Learning rate set to 0.198406
0:	learn: 1.3409635	total: 413ms	remaining: 3m 25s
1:	learn: 1.3135553	total: 776ms	remaining: 3m 13s
2:	learn: 1.2934919	total: 1.11s	remaining: 3m 3s
3:	learn: 1.2781852	total: 1.48s	remaining: 3m 3s
4:	learn: 1.2675226	total: 1.82s	remaining: 2m 59s
5:	learn: 1.2591117	total: 2.18s	remaining: 2m 59s
6:	learn: 1.2523081	total: 2.52s	remaining: 2m 57s
7:	learn: 1.2469831	total: 2.9s	remaining: 2m 58s
8:	learn: 1.2421410	total: 3.22s	remaining: 2m 55s
9:	learn: 1.2381647	total: 3.57s	remaining: 2m 54s
10:	learn: 1.2349293	total: 3.92s	remaining: 2m 54s
11:	learn: 1.2323102	total: 4.27s	remaining: 2m 53s
12:	learn: 1.2303441	total: 4.6s	remaining: 2m 52s
13:	learn: 1.2284166	total: 4.92s	remaining: 2m 50s
14:	learn: 1.2266211	total: 5.25s	remaining: 2m 49s
15:	learn: 1.2252021	total: 5.58s	remaining: 2m 48s
16:	learn: 1.2240305	total: 5.92s	remaining: 2m 48s
17:	learn: 1.2227737	total: 6.25s	remaining: 2m 47s
18:	learn: 1.2219326	total: 6.58s	remaining: 2m 

In [ ]:
dts = pd.read_csv('test.csv')
Xts = dts.copy()
del(Xts['Unnamed: 0'])
del(Xts['ID'])
Xts.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [ ]:
Xts = clean_dataset(Xts)
Xts.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,TRABAJO SOCIAL,BOLIVAR,1.0,1.0,3.0,1.0,7.0,3.0,1.0
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,4.0,3.0,2.0,1.0,5.0,7.0,0.0
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,3.0,0.0,3.0,1.0,4.0,5.0,0.0
3,20195,CONTADURIA PUBLICA,SUCRE,3.0,3.0,1.0,0.0,2.0,2.0,0.0
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,4.0,2.0,2.0,1.0,5.0,5.0,0.0


In [ ]:
Xts = transform_data(Xts)
Xts.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,2.577201,2.011725,2.282220,1.0,1.0,3.0,1.0,7.0,3.0,1.0
1,2.584049,2.193516,2.667522,4.0,3.0,2.0,1.0,5.0,7.0,0.0
2,2.394607,3.083426,2.557373,3.0,0.0,3.0,1.0,4.0,5.0,0.0
3,2.458686,2.194167,1.975351,3.0,3.0,1.0,0.0,2.0,2.0,0.0
4,2.394607,2.298972,2.454827,4.0,2.0,2.0,1.0,5.0,5.0,0.0


In [ ]:
preds = z.predict(Xts)
preds = preds.flatten()
preds

array([1, 3, 4, ..., 3, 4, 4])

In [ ]:
preds = pd.Series(preds,name='RENDIMIENTO_GLOBAL')
preds = preds.map({1: "bajo", 2: "medio-bajo", 3: "medio-alto", 4: "alto"})
preds

,RENDIMIENTO_GLOBAL
0,bajo
1,medio-alto
2,alto
3,bajo
4,bajo
...,...
296781,medio-bajo
296782,alto
296783,medio-alto
296784,alto


In [ ]:
#csv de prediccion
submission = pd.DataFrame([dts.ID,preds]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv('submission.csv',index=False)